In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
data = pd.read_csv("/home/iiita/Sentiment.csv", error_bad_lines=False)

In [3]:
data = data[["sentiment", "text"]]

In [5]:
data['text'] = data['text'].apply(lambda x: x.lower())

In [6]:
max_fatures = 2000
    

In [7]:
tokenizer = Tokenizer(num_words=max_fatures, split=' ')

In [8]:
tokenizer.fit_on_texts(data['text'].values)

In [9]:
X = tokenizer.texts_to_sequences(data['text'].values)

In [10]:
X = pad_sequences(X)

In [11]:
X

array([[   0,    0,    0, ...,   57, 1045,    1],
       [   0,    0,    0, ...,   11,    6,    7],
       [   0,    0,    0, ...,   14,  709,  510],
       ...,
       [   0,    0,    0, ...,   76,   69,    4],
       [   0,    0,    0, ..., 1529,   90,   45],
       [   0,    0,    0, ...,    6,    7,  894]], dtype=int32)

In [12]:
embed_dim = 128
lstm_out = 128

In [19]:
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [20]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 29, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 387,971
Trainable params: 387,971
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
Y = pd.get_dummies(data['sentiment']).values

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

In [23]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9293, 29) (9293, 3)
(4578, 29) (4578, 3)


In [24]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 2)

Epoch 1/15
 - 8s - loss: 0.8409 - acc: 0.6404
Epoch 2/15
 - 7s - loss: 0.7044 - acc: 0.7005
Epoch 3/15
 - 8s - loss: 0.6460 - acc: 0.7285
Epoch 4/15
 - 7s - loss: 0.6073 - acc: 0.7471
Epoch 5/15
 - 7s - loss: 0.5801 - acc: 0.7608
Epoch 6/15
 - 7s - loss: 0.5530 - acc: 0.7706
Epoch 7/15
 - 7s - loss: 0.5212 - acc: 0.7837
Epoch 8/15
 - 7s - loss: 0.4971 - acc: 0.7950
Epoch 9/15
 - 7s - loss: 0.4801 - acc: 0.8008
Epoch 10/15
 - 7s - loss: 0.4583 - acc: 0.8111
Epoch 11/15
 - 7s - loss: 0.4391 - acc: 0.8192
Epoch 12/15
 - 7s - loss: 0.4181 - acc: 0.8264
Epoch 13/15
 - 8s - loss: 0.4094 - acc: 0.8305
Epoch 14/15
 - 9s - loss: 0.3929 - acc: 0.8387
Epoch 15/15
 - 8s - loss: 0.3826 - acc: 0.8444


In [25]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

In [26]:
print("score", score)
print("accuracy", acc)

score 1.0548646715125853
accuracy 0.6507208387942333


In [51]:
twt = ['this is good']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
twt

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         24,   9, 146]], dtype=int32)

In [52]:
sentiment

array([0.09568974, 0.07328163, 0.8310286 ], dtype=float32)

In [ ]:
#negative / neutral / positive